In [1]:
from os import path
import numpy as np
import pandas as pd

from config import EMBEDDING_PATH
from emb_extr_res.emb_extr_res import get_embeddings_df
from load_data.wdc.load_wdc_dataset import EnglishDatasetLoader

In [7]:
dataset_type = "cameras"
dataset_size = "medium"

In [8]:
#load dataset
train_df = EnglishDatasetLoader.load_train(dataset_type, dataset_size)
test_df = EnglishDatasetLoader.load_test(dataset_type)

In [2]:
train_embeddings_path = path.join(EMBEDDING_PATH, r'train_embeddings.csv')
train_embeddings = get_embeddings_df(train_embeddings_path)

In [10]:
def extract_ids_and_cluster_ids(train_df):
    """
    extract unique pairs of cluster_id and offer_id
    """
    train_df_left = train_df[["id_left", "title_left"]]
    train_df_right = train_df[["id_right", "title_right"]]
    train_df_left = train_df_left.drop_duplicates().rename({"id_left" : "offer_id", "title_left" : "title"}, axis = 'columns')
    train_df_right = train_df_right.drop_duplicates().rename({"id_right" : "offer_id", "title_right" : "title"}, axis = 'columns')
    df_train_all = pd.concat([train_df_right, train_df_left])

    return df_train_all.drop_duplicates()

In [15]:
res = extract_ids_and_cluster_ids(train_df)

In [26]:
train_embeddings.shape

(2782, 769)

In [27]:
ids, counts = np.unique(train_embeddings["id"], return_counts=True)

In [29]:
sum(counts)

2782

In [12]:
train_embeddings.drop(["id"], axis=1).to_csv('train_embeddings.tsv', sep='\t', index=False, header=False)

In [21]:
pd.merge(train_embeddings, res, left_on = "id", right_on = "offer_id",  how='left')["title"].to_csv('train_embeddings_titles.tsv', sep='\t', index=False, header=False)#.rename({"product_id":"label"}, axis = 'columns')